<a href="https://colab.research.google.com/github/Sminjoo/Project_ORI-Team_drive_insurance/blob/main/modeling/SMJ_%EB%AA%A8%EB%8D%B8%EB%A7%81_(lightgbm_str_kfold).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/보험대회/train.csv')
test = pd.read_csv('/content/drive/MyDrive/보험대회/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/보험대회/sample_submission.csv')

In [ ]:
train2= pd.read_csv('/content/drive/MyDrive/보험대회/train2.csv')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train3= pd.concat([train, train2])

In [ ]:
train3.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [ ]:
gender = {'Male':0, 'Female':1}
vehicle_Age = {'< 1 Year' : 0, '1-2 Year' : 1, '> 2 Years' : 2}
vehicle_Damage = {'No' :0, 'Yes':1}

In [ ]:
for df_ls in [train3, test]:
  df_ls.replace({"Gender": gender}, inplace=True)
  df_ls.replace({"Vehicle_Age": vehicle_Age}, inplace=True)
  df_ls.replace({"Vehicle_Damage": vehicle_Damage}, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_scale = ['Age', 'Annual_Premium', 'Vintage']

train3[columns_to_scale] = scaler.fit_transform(train3[columns_to_scale])
test[columns_to_scale] = scaler.transform(test[columns_to_scale])

In [ ]:
train3["car_sum"] = train3["Vehicle_Damage"].astype("str") + train3["Driving_License"].astype("str") + train3["Previously_Insured"].astype("str")
train3["car_sum"] = train3["car_sum"].astype("category").cat.codes

test["car_sum"] = test["Vehicle_Damage"].astype("str") + test["Driving_License"].astype("str") + test["Previously_Insured"].astype("str")
test["car_sum"] = test["car_sum"].astype("category").cat.codes

In [ ]:
train3.head(5)

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,car_sum
0,0,0.015385,1,35.0,0,1,1,0.116218,124.0,0.612457,0,6
1,0,0.353846,1,28.0,0,2,1,0.104702,26.0,0.961938,1,6
2,1,0.076923,1,14.0,1,0,0,0.065880,152.0,0.844291,0,3
3,1,0.230769,1,1.0,0,1,1,0.000000,156.0,0.228374,0,6
4,1,0.246154,1,15.0,1,1,0,0.054547,152.0,0.982699,0,3


In [ ]:
from sklearn.model_selection import train_test_split
X = train3.drop('Response', axis=1)
y = train3['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
## 1) 검색 공간 설정

from hyperopt import hp

## LightGBM이 XBGoost에 비해 깊이가 깊음
lgbm_search_space = {
    'num_leaves':hp.quniform('num_leaves', 32, 64, 1),
    'max_depth':hp.quniform('max_depth', 100, 169, 1),                  ## 정수형 하이퍼 파라미터 => quniform 사용
    'min_child_weight':hp.quniform('min_child_weight', 60, 100, 1),     ## 정수형 하이퍼 파라미터 => quniform 사용
    'learning_rate':hp.uniform('learning_rate', 0.01, 0.2),
    'subsample':hp.uniform('subsample', 0.7, 1),
}

## 2) 목적 함수 설정

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

def objective_func(search_space):

  lgbm_clf = LGBMClassifier(
      n_estimators=100,
      max_depth=int(search_space['max_depth']),                ## int형으로 형변환 필요
      min_child_weight=int(search_space['min_child_weight']),  ## int형으로 형변환 필요
      learning_rate=search_space['learning_rate'],
      subsample=search_space['subsample'],
      num_leaves=int(search_space['num_leaves']),
      #eval_metric='logloss' => 불균형 데이터셋이므로 성능 평가 지표를 roc-auc로 설정
      )

      ## XGBoost와 LightGBM에서는 cross_val_score()를 적용하면 early stopping 지원 불가, KFold 방식으로 직접 구현해야 함
      ## 3개의 k-fold 방식으로 평가된 roc-auc 지표를 담는 list
  roc_auc_list = []

  ## 3개의 Stratified k-fold 방식 적용
  skf = StratifiedKFold(n_splits=3)

  for tr_index, val_index in skf.split(X_train, y_train):

    X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
    X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

    lgbm_clf.fit(
        X_tr, y_tr,
        eval_metric='auc',
        eval_set=[(X_tr, y_tr), (X_val, y_val)]
    )

    score = roc_auc_score(
        y_val,
        lgbm_clf.predict_proba(X_val)[:, 1]
    )

    roc_auc_list.append(score)

  return (-1) * np.mean(roc_auc_list)

## 3) fmin()을 사용하여 최적 하이퍼 파라미터 찾기
!pip install hyperopt
from hyperopt import fmin, tpe, Trials

trials = Trials()

best = fmin(
    fn=objective_func,
    space=lgbm_search_space,
    algo=tpe.suggest,
    max_evals=10,  ## 10번 만큼 반복하며 최적의 하이퍼 파라미터 찾음
    trials=trials,
)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Number of positive: 779694, number of negative: 5559456
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.469042 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 744
[LightGBM] [Info] Number of data points in the train set: 6339150, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122997 -> initscore=-1.964354
[LightGBM] [Info] Start training from score -1.964354
[LightGBM] [Info] Number of positive: 779693, number of negative: 5559457
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.445718 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 748
[LightGBM] [Info] Number of data points in the train set: 6339150, number of used features: 11
[LightGBM]

In [ ]:
## 획득한 최적의 하이퍼 파라미터를 이용하여 모델 선언
lgbm_wrapper = LGBMClassifier(
    n_estimators=500,
    num_leaves=int(best['num_leaves']),
    learning_rate=round(best['learning_rate'], 5),
    max_depth=int(best['max_depth']),
    min_child_weight=int(best['min_child_weight']),
    subsample=round(best['subsample'], 5)
)

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# 조기 중단 파라미터 설정
eval_set = [(X_train, y_train), (X_test, y_test)]
early_stopping_rounds = 20
best_score = -np.inf
best_iteration = 0
tolerance_rounds = 0

# 반복하면서 최적의 모델 찾기
for i in range(1, 20):  # 최대 20번까지 시도
    lgbm_wrapper = lgb.LGBMClassifier(n_estimators=i)
    lgbm_wrapper.fit(X_train, y_train)
    score = roc_auc_score(y_test, lgbm_wrapper.predict_proba(X_test)[:, 1])

    print(f"Iteration {i}, AUC score: {score}")

    if score > best_score:
        best_score = score
        best_iteration = i
        tolerance_rounds = 0
    else:
        tolerance_rounds += 1

    if tolerance_rounds >= early_stopping_rounds:
        break

print(f"Best iteration: {best_iteration}")
print(f"Best score: {best_score}")

[LightGBM] [Info] Number of positive: 1169540, number of negative: 8339185
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.647916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 748
[LightGBM] [Info] Number of data points in the train set: 9508725, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122997 -> initscore=-1.964355
[LightGBM] [Info] Start training from score -1.964355
Iteration 1, AUC score: 0.8541042110291986
[LightGBM] [Info] Number of positive: 1169540, number of negative: 8339185
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.396868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 748
[LightGBM] [Info] Number of data points in the train set: 95

In [ ]:
# 최적의 n_estimators로 모델 다시 학습
lgbm_wrapper.set_params(n_estimators=best_iteration)
lgbm_wrapper.fit(X_train, y_train, eval_set=eval_set, eval_metric='auc')

[LightGBM] [Info] Number of positive: 1169540, number of negative: 8339185
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.412962 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 748
[LightGBM] [Info] Number of data points in the train set: 9508725, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122997 -> initscore=-1.964355
[LightGBM] [Info] Start training from score -1.964355


LGBMClassifier(n_estimators=19)

In [ ]:
lgbm_roc_score = roc_auc_score(
    y_test,
    lgbm_wrapper.predict_proba(X_test)[:, 1]
)

lgbm_roc_score

0.8654752964212294

In [ ]:
y_pred = lgbm_wrapper.predict(X_test)

In [ ]:
import lightgbm as lgb
test_input = test.values
probabilities = lgbm_wrapper.predict_proba(test_input)

second_class_probabilities = probabilities[:, 1]
submission['Response']= second_class_probabilities
submission.to_csv("/content/drive/MyDrive/보험대회/submission_lightgbm_skf.csv", index=False)

In [ ]:
#피처중요도
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import plot_importance


ax = lgb.plot_importance(lgbm_wrapper, importance_type='split', max_num_features=10, figsize=(10, 6))
plt.title('Feature Importance (Split)')
plt.show()

In [ ]:
submission['Response'].value_counts()

Response
0.018704    3366291
0.041278     374105
0.021087      73513
0.020650      60475
0.020952      39612
             ...   
0.308792          1
0.377229          1
0.219295          1
0.295898          1
0.368247          1
Name: count, Length: 168625, dtype: int64